In [6]:
from pymatgen.core.composition import *
import numpy as np
import pandas as pd
import ase.db # https://wiki.fysik.dtu.dk/ase/ase/db/db.html
import json
import re

In [8]:
df_atoms = pd.read_csv("../datasets/project1/Schleder2019_AtomicTable.csv")

In [9]:
# All possible properties included in the database

keys=['folder', 'uid', 'plasmafrequency_x', 'plasmafrequency_y', 'asr_id', 'cell_area', 
      'has_inversion_symmetry', 'stoichiometry', 'spacegroup', 'spgnum', 'pointgroup', 
      'crystal_type', 'dos_at_ef_nosoc', 'dos_at_ef_soc', 'alphax_el', 'alphay_el', 'alphaz_el', 
      'ehull', 'hform', 'thermodynamic_stability_level', 'spin_axis', 'E_x', 'E_y', 'E_z', 'dE_zx', 
      'dE_zy', 'dim_primary', 'dim_primary_score', 'dim_nclusters_0D', 'dim_nclusters_1D', 'dim_nclusters_2D', 
      'dim_nclusters_3D', 'dim_threshold_0D', 'dim_threshold_1D', 'dim_threshold_2D', 'dim_score_0D', 
      'dim_score_1D', 'dim_score_2D', 'dim_score_3D', 'dim_score_01D', 'dim_score_02D', 'dim_score_03D', 
      'dim_score_12D', 'dim_score_13D', 'dim_score_23D', 'dim_score_012D', 'dim_score_013D', 'dim_score_023D', 
      'dim_score_123D', 'dim_score_0123D', 'first_class_material', 'minhessianeig', 'dynamic_stability_phonons', 
      'speed_of_sound_x', 'speed_of_sound_y', 'dynamic_stability_stiffness', 'c_11', 'c_12', 'c_13', 
      'c_21', 'c_22', 'c_23', 'c_31', 'c_32', 'c_33', 'magstate', 'is_magnetic', 'nspins', 'evac', 
      'evacdiff', 'dipz', 'efermi', 'gap', 'vbm', 'cbm', 'gap_dir', 'gap_dir_nosoc', 'gap_nosoc', 
      'workfunction', 'emass_vb_dir1', 'emass_vb_dir2', 'emass_cb_dir1', 'emass_cb_dir2', 
      'has_asr_hse_calculate', 'has_asr_plasmafrequency', 'has_asr_phonons_calculate', 
      'has_asr_database_material_fingerprint', 'has_asr_structureinfo', 'has_asr_pdos', 
      'has_asr_setup_strains', 'has_asr_polarizability', 'has_asr_convex_hull', 'has_asr_magnetic_anisotropy', 
      'has_asr_dimensionality', 'has_asr_setinfo', 'has_asr_phonons', 'has_asr_bader', 
      'has_asr_stiffness', 'has_asr_magstate', 'has_asr_gs_calculate', 'has_asr_gs', 
      'has_asr_bandstructure_calculate', 'has_asr_bandstructure', 'has_asr_projected_bandstructure', 
      'has_asr_pdos_calculate', 'has_asr_emasses_refine', 'has_asr_emasses', 'has_asr_emasses_validate']

data = ase.db.connect('../datasets/project1/c2db-2021-06-24.db')
data_nonMag = data.select(is_magnetic=False) # Creates a object with all the needed information. 
                                             # Excluding magnetic materials.

target='gap'
df_materials = pd.DataFrame(columns=['Material','Space Group',target,'Atom1', 'Atom2'])

for row in data_nonMag:
    try:
        formula_information = Composition(row.formula).as_dict()
        list_ele = list(formula_information.items())          
        
        if (len(list_ele)==2 and list_ele[0][1]==1 and list_ele[1][1]==2): # Only AB2 stoichiometry

            new_entry = [row.formula, row.spacegroup, row[target], list_ele[0][0], list_ele[1][0]]            
            df_materials.loc[len(df_materials)] = new_entry

    except:
        pass

df_materials

,Material,Space Group,gap,Atom1,Atom2
0,AgBr2,P-4m2,0.000000,Ag,Br
1,AgLi2,P-6m2,0.000000,Ag,Li
2,AlI2,P-6m2,0.000000,Al,I
3,AsCl2,P-6m2,0.000000,As,Cl
4,AsS2,P-3m1,0.000000,As,S
...,...,...,...,...,...
620,ZrI2,P-6m2,0.687104,Zr,I
621,ZrO2,P-4m2,4.368627,Zr,O
622,ZrS2,P-3m1,1.159195,Zr,S
623,ZrSe2,P-3m1,0.336550,Zr,Se


In [11]:
df_atoms.query('Element == "Ag" or Element == "Br"')

,Element,Z,Electronegativity,IonizationPotential,ElectronAffinity,HOMO,LUMO,r_s_orbital,r_p_orbital,r_d_orbital,r_atomic_nonbonded,r_valence_lastorbital,r_covalent,Valence,PeriodicColumn,PeriodicColumn_upto18,NumberUnfilledOrbitals,Polarizability
34,Br,35,2.96,-12.5044,-3.5150,-7.8585,5.4193,0.7514,0.8834,0.2281,1.14,0.8834,1.20,17.0,7.0,17.0,1.0,21.8
46,Ag,47,1.93,-7.8182,-1.2721,-4.4022,0.9657,1.3241,1.4180,0.5486,1.53,1.3241,1.45,11.0,1.0,11.0,1.0,52.5
